In [ ]:
import gzip
from xml.dom import minidom
from utils.list_files import get_stem
import os
import json

In [ ]:
with gzip.open('data/Siren_063021_NoPec_MoreReverb_SideEntrance+Lighting.als') as f:
    raw = minidom.parseString(f.read())
    
def get_attribute(elt, attrib_name):
    return elt.attributes[attrib_name].value
    
def get_tag_attribute(elt, tag_name, attrib_name):
    target = elt
    for e in tag_name.split('/'):
        target = target.getElementsByTagName(e)[0]
    return get_attribute(target, attrib_name)

In [ ]:
tempo = float(get_tag_attribute(raw, 'Tempo/Manual', 'Value'))
seconds_per_beat = 60 / tempo

In [ ]:
# get all arranger audio clips and flatten 
all_automation = raw.getElementsByTagName('ArrangerAutomation')
audio_clips = [e.getElementsByTagName('AudioClip') for e in all_automation]
audio_clips = [e for l in audio_clips for e in l]

In [ ]:
offsets = []

for audio_clip in audio_clips:
    try:
        # live 10
        filename = get_tag_attribute(audio_clip, 'FileRef/Name', 'Value')
    except:
        # live 11
        filename = get_tag_attribute(audio_clip, 'FileRef/RelativePath', 'Value')
        _, filename = os.path.split(filename)
    start = get_tag_attribute(audio_clip, 'CurrentStart', 'Value')
    start = float(start) * seconds_per_beat # beats
    end = get_tag_attribute(audio_clip, 'CurrentEnd', 'Value')
    end = float(end) * seconds_per_beat # beats
    loop_start = get_tag_attribute(audio_clip, 'LoopStart', 'Value')
    loop_start = float(loop_start) # seconds
    
    # patch this filename with the other, which is offset
    if filename == '022619_Solo with Shrimp_Bass_mcq_02-28-20_Start Theme 1.wav':
        filename = '022619_Solo with Shrimp_Bass_mcq_02-28-20.wav'
        loop_start += 7 * 60 + 4 # 7m4s from start of original
        
    offsets.append((filename, loop_start, start, end))
    
# dedupe to remove overlapping tracks
offsets = list(set(offsets))

In [ ]:
len(offsets), offsets

In [ ]:
output_framerate = 60

In [ ]:
sr = 44100
hop_length = 512
feature_framerate = sr / hop_length
feature_framerate

In [ ]:
import numpy as np

use_annotated = False

n_lights = 15
color_channels = 6

def is_annotated(stem):
    annotation_fn = f'data/{stem}.Table.1.selections.txt'
    return os.path.exists(annotation_fn)

end_time = max([e[-1] for e in offsets])
max_index = int(end_time * output_framerate)
full_design = np.zeros((max_index, n_lights, color_channels))

out_of_bounds = 0

for audio_fn, loop_start, start, end in offsets:
    stem = get_stem(audio_fn)
#     print(stem)

    if use_annotated != 'both' and is_annotated(stem) != use_annotated:
        print('skipping', stem)
        continue
    print('using', stem)
    
    design_fn = f'design/{stem}.npy'
    if not os.path.exists(design_fn):
        print(start, end, 'skipping', stem)
        continue

    design = np.load(design_fn)
    
    start_index = int(start * output_framerate)
    end_index = int(end * output_framerate)
    
    # this uses a nearest neighbor interpolation
    # but we could use another technique for rescaling
    for i in range(start_index, end_index):
        t = (i - start_index) / output_framerate
        t += loop_start
        j = int(t * feature_framerate)
        
        try:
            full_design[i] += design[j]
        except:
            out_of_bounds += 1
        
print()
print('max', full_design.max())
print('out of bounds', out_of_bounds)

dmx = (np.clip(full_design, 0, 1) * 255).astype(int)

output = {
    'framerate': output_framerate,
    'data': dmx.tolist()
}

if use_annotated == 'both':
    description = ''
else:
    description = '-annotated' if use_annotated else '-unannotated'
fn = f'design{description}.json'
print('saving', fn)
with open(fn, 'w') as f:
    json.dump(output, f, separators=(',',':'))#, indent=2)

print('done')